In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

In [ ]:
# 引数：inputDF  はreturnRawDF()の返り値
# 引数：repeated はDATA列を繰り返す回数
# 引数：includeLastData は学習に最後の値を使うかどうかを指定

def generateInputFileForExtraP(inputDF = returnRawDF(fix="Class"), repeated = 3, includeLastData = False, fileName = "tmp_functionName.txt"):

    # https://github.com/extra-p/extrap/blob/master/docs/examples/input.txt
    # が入力例のテキストファイルとなっており、これに対応するファイルを作成する。
    functionNames = inputDF.index.tolist()
    for functionName in functionNames:
        tmpfile = open(fileName, 'w+')
        columns = inputDF.columns.tolist()
        tmpfile.write("PARAMETER p\n")
        tmpfile.write("POINTS")
        
        if(includeLastData == False):
            columns = columns[:-1]
        
        for column in columns:
            tmpfile.write(f" ({column})")
        tmpfile.write("\n\n")

        tmpfile.write(f"REGION {functionName}\n")
        tmpfile.write("METRIC functionCall\n")

        for column in columns:

            tmpfile.write("DATA")
            content = float(inputDF.at[functionName, column])

            for i in range(repeated):
                tmpfile.write(f" {content}")
            tmpfile.write("\n")

        tmpfile.close()